# Measurement conversion

In [87]:
import pandas as pd
import numpy as np
import spacy

import sys
sys.path.append("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta")
import functions

import requests

In [59]:
posts1 = pd.read_csv("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta/Instagram Data Scraping/Phantom Buster/recipe_posts.csv")
posts2 = pd.read_csv("/Users/maxkirwan/Desktop/Uni/Data Science MSc/Data Science Project/nutrition-insta/Instagram Data Scraping/Phantom Buster/recipe_posts_2.csv")

posts = pd.concat([posts1,posts2])

In [60]:
# Getting english posts
posts = functions.get_english_posts(posts)

Detecting language of each post...
Language detection complete.
Time taken: 0:04:54.716132


In [61]:
# Preprocessing text descriptions
posts['description_preprocessed'] = posts['description'].apply(functions.preprocess_text)

In [62]:
def includes_ingredient_list(text):
    
    text_str = str(text)
    if ("ingredients:" in text or "ingredients :" in text) and text_str.count('\n') > 8:
        return True
        
    else:
        return False
    
# Adding includes_ingredient_list binary column
posts['includes_ingredient_list'] = posts['description_preprocessed'].apply(includes_ingredient_list)
# Keeping only posts with ingredient_list
posts = posts[posts['includes_ingredient_list']]

In [63]:
nlp = spacy.load("./model-best")

# Merging entities into single tokens
nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [64]:
def get_ingredient_triplets(text):
    
    '''
    Function to identify entity triplets which relate to unique ingredients.
    These triplets come in the form consecutive QUANTITY, MEASUREMENT, INGREDIENT entities.
    '''
    
    doc = nlp(text)
    ingredient_triplets = []
    
    for i in range(len(doc)-10):
        
        if doc[i].ent_type_ == 'QUANTITY' and doc[i+1].ent_type_ == 'MEASUREMENT':
            
            quantity_index = i
            measurement_index = i+1
                
            for j in range(i+1,i+8):

                if doc[j].ent_type_ == 'INGREDIENT':

                    ingredient_index = j
                    ingredient_triplets.append((quantity_index,measurement_index,ingredient_index))
                    break
                    
                    
    def ingredient_triplets_to_list(triplets):

        dict_list = []

        for i, j, k in triplets:

            ing_dict = {}

            ing_dict['ingredient'] = doc[k]
            ing_dict['quantity'] = doc[i]
            ing_dict['measurement'] = doc[j]

            dict_list.append(ing_dict)

        return dict_list
        
    return ingredient_triplets_to_list(ingredient_triplets)

In [65]:
# Getting ingredient triplets for all posts
posts['ingredient_list'] = posts['description_preprocessed'].apply(get_ingredient_triplets)

In [74]:
# function to convert string fractions to float, e.g '1\4'->0.25
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        try:
            num, denom = frac_str.split('/')
        except ValueError:
            # If unknown format just return 1
            return 1.0
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [80]:
# getting list of unqiue measurements
measurements = []
for ing_list in posts['ingredient_list']:
    for ing in ing_list:
        measurements.append(str(ing['measurement']))
pd.Series(measurements).unique()

array(['cup', 'teaspoon', 'tablespoon', 'gram', 'litre', 'clove', 'ounce',
       'heap', 'g', 'jars', 'scoops', 'bunch', 'dash', 'scp', 'kg',
       'glass', 'inch', 'plum', 'handfuls', 'lemon', 'slit', 'packet',
       'cinnamon', 'package', 't.', 'peaches', 'c.', 'boxes', 'container',
       'block', '-inch', 'blackberries', 'bell', 'roma', 'jar', 'pint',
       'red', 'strip', '⅛-inch'], dtype=object)

In [81]:
def convert_measurement_to_grams(ing_list):
    
    '''
    Take as input list of ingredient triplets.
    Return list of (ingredient, quantity in grams) tuples.
    '''
    
    ing_quan_list = []
    
    for ing in ing_list:
        
        if str(ing['measurement'])=='cup':
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*200)
            
        elif str(ing['measurement']) in ['teaspoon','dash']:
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*4.5)
            
        elif str(ing['measurement']) in ['tablespoon','heap','scoop','scp']:
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*12.5)
            
        elif str(ing['measurement']) in ['gram','g','ml']:
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity'])))
            
        elif str(ing['measurement']) in ['litre', 'kg']:
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*1000)
        
        elif str(ing['measurement'])=='cl':
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*10)
            
        elif str(ing['measurement'])=='ounce':
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*28)
            
        elif str(ing['measurement']) in ['jar','jars']:
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*300)
            
        elif str(ing['measurement'])=='pint':
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*473)
            
        elif str(ing['measurement']) in ['packet','package','container']:
            ing_quan = (str(ing['ingredient']), convert_to_float(str(ing['quantity']))*200)
        
        # If unknown quantity then default to 100 grams
        else:
            ing_quan = (str(ing['ingredient']), 100.0)
            
        ing_quan_list.append(ing_quan)
        
    return ing_quan_list

In [82]:
# Applying function to df to get column with (ingredient,quantitiy in grams) tuple
posts['ingredients_grams'] = posts['ingredient_list'].apply(convert_measurement_to_grams)

## Nutritional Information
USDA lookup for nutritional info.

In [84]:
def get_usda_info_json(ingredient):
    
    headers = {}
    params = {'api_key': 'XBZBkznaG3lFCfED92u489nCZFlm9zdBNdH6lCEZ'}
    
    json_data = {
        'query': ingredient,
        'dataType': [
            'Foundation',
            'SR Legacy'
        ],
        'pageSize' : 1,
        'pageNumber' : 1,
        'sortBy': 'dataType.keyword',
        'sortOrder': 'asc',
    }

    response = requests.post('https://api.nal.usda.gov/fdc/v1/foods/list', params=params, headers=headers, json=json_data)
    
    return response.json()

In [112]:
def calc_nutritional_info_recipe(ing_quan_list):
    '''
    USDA food lookup for each ingredient in recipe.
    Then totals levels of required nutrients
    (including calculating levels in ingredient by 100 gram conversion calculation)
    To give total nutrient levels for whole recipe.
    '''
    
    ENERGY = 0     # KCAL
    PRO = 0        # G
    CHO = 0        # G
    FAT = 0        # G
    
    
    for ing, quan in ing_quan_list:
        
        # Query ingredient against FoodData Central database
        response = get_usda_info_json(ing)

        
        # If query returned a response
        if response:
            
            # If query caused an error
            try:
                error_message = response['error']
                break
            except:
        
                # Get list of nutrients
                nutrients = response[0]['foodNutrients']

                for nutrient in nutrients:

                    if nutrient['name'] == 'Total lipid (fat)':
                        FAT += nutrient['amount'] * quan / 100
                        print("added",ing,'//',nutrient['name'],nutrient['amount'])

                    elif nutrient['name'] == 'Carbohydrate, by difference':
                        CHO += nutrient['amount'] * quan / 100
                        print("added",ing,'//',nutrient['name'],nutrient['amount'])

                    elif nutrient['name'] == 'Protein':
                        PRO += nutrient['amount'] * quan / 100
                        print("added",ing,'//',nutrient['name'],nutrient['amount'])

                    elif (nutrient['name'] == 'Energy' and nutrient['unitName'] == 'KCAL') or nutrient['name'] == 'Energy (Atwater General Factors)':
                        ENERGY += nutrient['amount'] * quan / 100
                        print("added",ing,'//',nutrient['name'],nutrient['amount'])
                
                
    nutrient_levels = {'ENERGY': ENERGY,
                       'PRO' : PRO,
                       'CHO' : CHO,
                       'FAT' : FAT
                      }
    
    return nutrient_levels



def convert_dict_to_series_values(nutrient_levels_dict):
    
    return pd.Series(nutrient_levels_dict.values())

In [89]:
calc_nutritional_info_recipe(posts['ingredients_grams'].iloc[0])

added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added milk // Protein 7.81
added milk // Total lipid (fat) 11.0
added milk // Carbohydrate, by difference 6.86
added milk // Energy 157
added water // Total lipid (fat) 0.94
added water // Energy 90.0
added water // Protein 19.0
added water // Carbohydrate, by difference 0.08
added sugar // Protein 0.0
added sugar // Total lipid (fat) 0.32
added sugar // Energy 385
added sugar // Carbohydrate, by difference 99.6
added peanut butter // Total lipid (fat) 49.4
added peanut butter // Protein 24.0
added peanut butter // Carbohydrate, by difference 22.7
added peanut butter // Energy (Atwater General Factors) 632
added banana // Total lipid (fat) 0.22
added banana // Protein 0.73
added banana // Carbohydrate, by difference 20.1
added banana // Energy 85.0


{'ENERGY': 742.95, 'PRO': 43.10125, 'CHO': 86.67200000000001, 'FAT': 24.4669}

In [118]:
# Adding cols to df for energy, prot, cho, fat

nutrient_cols = ['ENERGY','PROT','CHO','FAT']

posts[nutrient_cols] = posts['ingredients_grams'].apply(calc_nutritional_info_recipe).apply(convert_dict_to_series_values)

added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added milk // Protein 7.81
added milk // Total lipid (fat) 11.0
added milk // Carbohydrate, by difference 6.86
added milk // Energy 157
added water // Total lipid (fat) 0.94
added water // Energy 90.0
added water // Protein 19.0
added water // Carbohydrate, by difference 0.08
added sugar // Protein 0.0
added sugar // Total lipid (fat) 0.32
added sugar // Energy 385
added sugar // Carbohydrate, by difference 99.6
added peanut butter // Total lipid (fat) 49.4
added peanut butter // Protein 24.0
added peanut butter // Carbohydrate, by difference 22.7
added peanut butter // Energy (Atwater General Factors) 632
added banana // Total lipid (fat) 0.22
added banana // Protein 0.73
added banana // Carbohydrate, by difference 20.1
added banana // Energy 85.0
added pesto // Protein 4.16
added pesto // Total lipid (fat) 36.4
added pesto /

added black pepper // Total lipid (fat) 0.106
added black pepper // Protein 0.715
added black pepper // Carbohydrate, by difference 4.78
added black pepper // Energy (Atwater General Factors) 22.9
added oats // Total lipid (fat) 6.31
added oats // Carbohydrate, by difference 69.9
added oats // Protein 13.2
added oats // Energy (Atwater General Factors) 389
added chia seeds // Total lipid (fat) 56.1
added chia seeds // Carbohydrate, by difference 17.1
added chia seeds // Energy 612
added chia seeds // Protein 21.0
added vanilla // Total lipid (fat) 0.06
added vanilla // Carbohydrate, by difference 12.6
added vanilla // Energy 288
added vanilla // Protein 0.06
added water // Total lipid (fat) 0.94
added water // Energy 90.0
added water // Protein 19.0
added water // Carbohydrate, by difference 0.08
added zucchini // Carbohydrate, by difference 3.11
added zucchini // Energy 21.0
added zucchini // Protein 2.71
added zucchini // Total lipid (fat) 0.4
added cashews // Protein 18.2
added cash

added lemon juice // Protein 0.55
added lemon juice // Total lipid (fat) 0.7
added lemon juice // Carbohydrate, by difference 7.59
added lemon juice // Energy 37.0
added tahini // Total lipid (fat) 53.0
added tahini // Protein 17.4
added tahini // Carbohydrate, by difference 21.5
added tahini // Energy 592
added parsley // Carbohydrate, by difference 6.33
added parsley // Energy 36.0
added parsley // Total lipid (fat) 0.79
added parsley // Protein 2.97
added salt // Total lipid (fat) 82.2
added ginger // Total lipid (fat) 0.75
added ginger // Protein 1.82
added ginger // Carbohydrate, by difference 17.8
added ginger // Energy 80.0
added chia seeds // Total lipid (fat) 56.1
added chia seeds // Carbohydrate, by difference 17.1
added chia seeds // Energy 612
added chia seeds // Protein 21.0
added pumpkin seeds // Total lipid (fat) 56.1
added pumpkin seeds // Carbohydrate, by difference 17.1
added pumpkin seeds // Energy 612
added pumpkin seeds // Protein 21.0
added butter // Total lipid (

added coriander // Total lipid (fat) 17.8
added coriander // Carbohydrate, by difference 55.0
added coriander // Energy 298
added coriander // Protein 12.4
added garlic powder // Total lipid (fat) 0.38
added garlic powder // Protein 6.62
added garlic powder // Carbohydrate, by difference 28.2
added garlic powder // Energy 143
added coriander // Total lipid (fat) 17.8
added coriander // Carbohydrate, by difference 55.0
added coriander // Energy 298
added coriander // Protein 12.4
added rice // Protein 7.19
added rice // Total lipid (fat) 3.85
added rice // Carbohydrate, by difference 75.5
added rice // Energy 365
added coriander // Total lipid (fat) 17.8
added coriander // Carbohydrate, by difference 55.0
added coriander // Energy 298
added coriander // Protein 12.4
added milk // Protein 7.81
added milk // Total lipid (fat) 11.0
added milk // Carbohydrate, by difference 6.86
added milk // Energy 157
added sugar // Protein 0.0
added sugar // Total lipid (fat) 0.32
added sugar // Energy 3

added chili flakes // Energy 349
added chili flakes // Total lipid (fat) 0.46
added chili flakes // Carbohydrate, by difference 83.3
added chili flakes // Protein 8.95
added sugar // Protein 0.0
added sugar // Total lipid (fat) 0.32
added sugar // Energy 385
added sugar // Carbohydrate, by difference 99.6
added oregano // Total lipid (fat) 4.28
added oregano // Carbohydrate, by difference 68.9
added oregano // Energy 265
added oregano // Protein 9.0
added chicken // Protein 23.9
added chicken // Total lipid (fat) 5.95
added chicken // Carbohydrate, by difference 0.0
added chicken // Energy 156
added butter // Total lipid (fat) 53.0
added butter // Carbohydrate, by difference 21.2
added butter // Protein 20.8
added butter // Energy (Atwater General Factors) 645
added tomato // Total lipid (fat) 0.425
added tomato // Protein 0.696
added tomato // Carbohydrate, by difference 3.84
added tomato // Energy (Atwater General Factors) 22.0
added tomato // Total lipid (fat) 0.425
added tomato // 

...

added garam masala // Total lipid (fat) 0.88
added garam masala // Carbohydrate, by difference 10.5
added garam masala // Energy 57.0
added garam masala // Protein 3.3
added cashews // Protein 18.2
added cashews // Total lipid (fat) 43.8
added cashews // Carbohydrate, by difference 30.2
added cashews // Energy 553
added milk // Protein 7.81
added milk // Total lipid (fat) 11.0
added milk // Carbohydrate, by difference 6.86
added milk // Energy 157
added yogurt // Protein 10.3
added yogurt // Total lipid (fat) 0.37
added yogurt // Carbohydrate, by difference 3.64
added yogurt // Energy 61.0
added rice // Protein 7.19
added rice // Total lipid (fat) 3.85
added rice // Carbohydrate, by difference 75.5
added rice // Energy 365
added ghee // Protein 0.0
added ghee // Total lipid (fat) 100
added ghee // Carbohydrate, by difference 0.0
added ghee // Energy 900
added garlic // Total lipid (fat) 0.38
added garlic // Protein 6.62
added garlic // Carbohydrate, by difference 28.2
added garlic // E

added tomatoes // Total lipid (fat) 0.425
added tomatoes // Protein 0.696
added tomatoes // Carbohydrate, by difference 3.84
added tomatoes // Energy (Atwater General Factors) 22.0
added water // Total lipid (fat) 0.94
added water // Energy 90.0
added water // Protein 19.0
added water // Carbohydrate, by difference 0.08
added salt // Total lipid (fat) 82.2
added cashew // Protein 18.2
added cashew // Total lipid (fat) 43.8
added cashew // Carbohydrate, by difference 30.2
added cashew // Energy 553
added yogurt // Protein 10.3
added yogurt // Total lipid (fat) 0.37
added yogurt // Carbohydrate, by difference 3.64
added yogurt // Energy 61.0
added cinnamon // Total lipid (fat) 5.29
added cinnamon // Carbohydrate, by difference 44.4
added cinnamon // Energy 253
added cinnamon // Protein 7.05
added cashew // Protein 18.2
added cashew // Total lipid (fat) 43.8
added cashew // Carbohydrate, by difference 30.2
added cashew // Energy 553
added ghee // Protein 0.0
added ghee // Total lipid (fat

added cinnamon // Total lipid (fat) 5.29
added cinnamon // Carbohydrate, by difference 44.4
added cinnamon // Energy 253
added cinnamon // Protein 7.05
added pepper // Total lipid (fat) 0.106
added pepper // Protein 0.715
added pepper // Carbohydrate, by difference 4.78
added pepper // Energy (Atwater General Factors) 22.9
added garlic // Total lipid (fat) 0.38
added garlic // Protein 6.62
added garlic // Carbohydrate, by difference 28.2
added garlic // Energy 143
added chili powder // Total lipid (fat) 14.3
added chili powder // Carbohydrate, by difference 49.7
added chili powder // Energy 282
added chili powder // Protein 13.5
added milk // Protein 7.81
added milk // Total lipid (fat) 11.0
added milk // Carbohydrate, by difference 6.86
added milk // Energy 157
added milk // Protein 7.81
added milk // Total lipid (fat) 11.0
added milk // Carbohydrate, by difference 6.86
added milk // Energy 157
added salt // Total lipid (fat) 82.2
added yeast // Total lipid (fat) 0.9
added yeast // Ca

added avocado // Protein 0.0
added avocado // Total lipid (fat) 100
added avocado // Carbohydrate, by difference 0.0
added avocado // Energy 884
added garlic // Total lipid (fat) 0.38
added garlic // Protein 6.62
added garlic // Carbohydrate, by difference 28.2
added garlic // Energy 143
added peanut butter // Total lipid (fat) 49.4
added peanut butter // Protein 24.0
added peanut butter // Carbohydrate, by difference 22.7
added peanut butter // Energy (Atwater General Factors) 632
added chili sauce // Protein 1.44
added chili sauce // Carbohydrate, by difference 6.74
added chili sauce // Energy 29.0
added chili sauce // Total lipid (fat) 0.19
added basil // Total lipid (fat) 0.64
added basil // Carbohydrate, by difference 2.65
added basil // Energy 23.0
added basil // Protein 3.15
added vanilla // Total lipid (fat) 0.06
added vanilla // Carbohydrate, by difference 12.6
added vanilla // Energy 288
added vanilla // Protein 0.06
added baked beans // Protein 24.4
added baked beans // Tota

added salt // Total lipid (fat) 82.2
added onion // Protein 0.94
added onion // Total lipid (fat) 0.1
added onion // Carbohydrate, by difference 9.93
added onion // Energy 44.0
added garlic // Total lipid (fat) 0.38
added garlic // Protein 6.62
added garlic // Carbohydrate, by difference 28.2
added garlic // Energy 143
added black pepper // Total lipid (fat) 0.106
added black pepper // Protein 0.715
added black pepper // Carbohydrate, by difference 4.78
added black pepper // Energy (Atwater General Factors) 22.9
added cucumber // Total lipid (fat) 0.43
added cucumber // Carbohydrate, by difference 1.99
added cucumber // Protein 0.48
added cucumber // Energy 12.0
added lemon juice // Protein 0.55
added lemon juice // Total lipid (fat) 0.7
added lemon juice // Carbohydrate, by difference 7.59
added lemon juice // Energy 37.0
added lettuce // Carbohydrate, by difference 3.24
added lettuce // Energy 17.0
added lettuce // Total lipid (fat) 0.26
added lettuce // Protein 1.24
added cheddar //

In [120]:
posts.to_csv("posts_with_macro_nutrients_1.csv")